In [ ]:
##This script finds minimum of a current trace at a specified potential
## Minimum can be calculated either by calculating the mean or by fitting 
# an biexpotential function in a specified time intervall of the recording

In [ ]:
import HEKAimport
#import glob
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
import scipy

In [ ]:
def loadHEKAFolder(filename):

    column_names = ['Filename', 'Group', 'Group_Index', 'Series', 'Series_Index']
    directory_df = pd.DataFrame(columns=column_names)


    for filename in glob.glob(filename):
        try:
            datafile = HEKAimport.HEKAfile(filename)
            for group in datafile.get_Groups():
                for series in group.Series:
                    directory_df.loc[len(directory_df)] = [filename, group.Label.split("\x00")[0], group.GroupCount, series.Label.split("\x00")[0], series.SeriesCount]





        except Exception as inst:
            print("Could not open "+filename)
            print(type(inst))
            print(inst.args)

    return directory_df 

In [ ]:
def biexponential(x, a, b, m, n, c):
    return a*np.exp(-x/b) + m*np.exp(-x/n) + c
#x_dummy = np.linspace(start=0.001, stop=0.5, num=5000)

In [ ]:
## fit an exponential equation to selected data (between time points start and stop) 
# to obtain minimum "I_pA" value for IV curve 
## creates a plot of the raw data and obtained function (if calc_mean=False and fitting was successful)
## If data can not be fitted (ore calc_mean=True), a mean value is calculated between time points start and stop
## returns I_pA, potential, unique name, animal, information about fit parameter
def fit_exp_get_min(directory_df, start=0, stop=14000, step=1, indices=[], pot=[], func=biexponential,\
                    calc_mean=False, bin=False, color="black"):
    #contains all information all selected traces
    data_IV = pd.DataFrame()
    #contains all information for each trace
    info={}
    
    # set bounds and  initial values if biexponental fit is selceted
    if func==biexponential:
        bounds=([0.0001, 0.0001,-300,0.01,-100],[np.inf,np.inf,-1,5,100])
        p0=[15, 0.01, -80, 0.04, -15]
        info["fit_func"] = "biexponential"

  
    
    # dummy data for plotting fit function
    x_dummy = np.linspace(start=0.001, stop=0.5, num=5000)
    
    # set list of indices per default to all indices
    if len(indices) == 0:
        indices = directory_df.index
    
    for i in indices:
        
        analysis_min = 0.015 # time point needed for generation of potential list 
        analysis_max = 0.025 #time point needed for generation of potential list 
        
        # select row
        selection = directory_df.query('index =='+str(i)+'')
        title='_'.join(map(str,(selection.iloc[0].values))).replace(".dat","").replace("data_10mMCa\\","").replace("_Cacna1hKO","")
        
        # set V_min, V_max
        V_min = selection["V_min"].values[0]
        V_max = selection["V_max"].values[0]
        
        # load original data file
        if (V_min != "n" or V_max != "n"):
            datafile = HEKAimport.HEKAfile(selection["Filename"].values[0])
            data = datafile.get_Sweeps_byindex(int(selection["Group_Index"].values[0])-1,\
                                               int(selection["Series_Index"].values[0])-1, ZeroOffset=True)
            # drop "Leak_1" data, only keep "I-mon" data
            data_I = data.drop(['Leak_1'], axis=1)

            # generate list of potentials
            a_series = pd.Series(np.asarray(datafile.get_IV(data[analysis_min:analysis_max]["I-mon"].idxmin().tolist(),\
                                int(selection["Group_Index"].values[0])-1, int(selection["Series_Index"].values[0])-1)),\
                                 index = data_I.columns, name="V")
            
            data_I = data_I.append(np.around(a_series,3), ignore_index=False)
            
            data_I.columns = data_I.iloc[len(data_I)-1]
            data_I = data_I.drop(data_I.index[len(data_I)-1])
            
            if len(pot) == 0:
                pot = data_I.columns
                
            for j in pot:
                if V_min <= j<= V_max:

                    info["Filename"] = selection["Filename"].values[0]
                    info["Group"] = selection["Group"].values[0]
                    info["Group_Index"] = selection["Group_Index"].values[0]
                    info["Series"] = selection["Series"].values[0]
                    info["Series_Index"] = selection["Series_Index"].values[0]
                    info["animal"] = selection["Filename"].values[0][29:36]
                    info["Name"] = info["animal"]+"_"+info["Group"]+\
                    "_"+str(info["Group_Index"])+\
                    "_"+str(info["Series_Index"])+\
                    "_"+info["Series"]

                    print(f">>>{title}: {j}")
                    y_data=data_I[j][start:stop:step].values
                    x_data=data_I[j][start:stop:step].index.values
                    
                    info["raw_data"] = data_I[j]
                    info["start"] = start
                    info["stop"] = stop
                    info["step"] = step

                    #fit data
                    if calc_mean==False:
                        popt, pcov = scipy.optimize.curve_fit(func, x_data,\
                                                                      y_data, p0=p0, maxfev = 100000, bounds=bounds)

                        minimum = scipy.optimize.fmin(func, 0.05, args=tuple(popt))
                    print("-------")
                    
                    # create plot
                    fig = plt.figure(figsize=(8,4))
                    plt.title(f">>>{title}: {j}")
                    if bin==False:
                        plt.plot(data_I[j][0:14000:step])
                        plt.xlim(-0.01,0.28)
                        plt.ylim(-100,50)
                    elif bin ==True:
                        binned=pd.DataFrame()
                        for b in range(0, 30500, 50):
                            indx = data_I[j].index[b]
                            mean = data_I[j][b:b+50].mean()
                            a = {'indx': [indx], 'mean': [mean]}
                            d = pd.DataFrame(data=a)
                            binned = pd.concat([binned,d], axis=0)
                        plt.plot(binned["indx"],binned["mean"], color=color, lw=0.2)
                        plt.xlim(-0.01,0.28)
                        plt.ylim(-100,50)
                        
                    #plot resulting function
                    if calc_mean==False:
                        y_fit = func(x_dummy, *popt)
                        plt.scatter(x_dummy, y_fit, s=10, color="orange",zorder=3)

                    
                    #calculate mean instead of fitting data
                    if calc_mean==True:
                        mean=y_data.mean()
                        plt.axvline(x=x_data[0], color="green")
                        plt.axvline(x=x_data[-1], color="green")
                        plt.axhline(y=mean, color="red", ls="--", zorder=5)
                        plt.text(0.28,mean, f"mean: {mean} pA.....{j} V", fontsize= 20 )
                        print(f"mean= {mean}")
                        info["I_pA"] = mean
                        info["mean"] = mean
                        info["fit_popt"] = "n"
                    
                    else:
                        
                        # if not minimum can be found in fit function calculate mean
                        if np.isinf(func(minimum[0], *popt)) or func(minimum[0], *popt) < -200:
                            mean=y_data[0:int(5000/step)].mean()
                            plt.axvline(x=x_data[0], color="green")
                            plt.axvline(x=x_data[int(5000/step)], color="green")
                            plt.axhline(y=mean, color="red", ls="--", zorder=5)
                            plt.text(0.28,mean, f"mean: {mean} pA.....{j} V", fontsize= 20 )
                            print(f"mean= {mean}")
                            info["I_pA"] = mean
                            info["mean"] = mean
                            info["fit_popt"] = "n"

                        # minimum was successfully determined 
                        else:
                            plt.axvline(x=x_data[0], color="green")
                            plt.axvline(x=x_data[-1], color="green")
                            plt.axvline(x=minimum[0],color = "red", zorder=4, ls="--")
                            plt.axhline(y=func(minimum[0], *popt),color = "red", zorder=5, ls="--")
                            plt.text(0.28,func(minimum[0], *popt), f"{func(minimum[0], *popt)} pA.....{j} V", fontsize= 20 )
                            print(f"minimum= {func(minimum[0], *popt)}")
                            info["I_pA"] = func(minimum[0], *popt)
                            info["mean"] = "n"
                            info["fit_popt"] = popt



                    info["V_V"] = j
                    pd.concat([data_IV, pd.Series(info)], ignore_index=True)
                    

        else:
            print(f"no analysis: {title}" )
            
    return data_IV

In [ ]:
directory_df=pd.read_excel("directory_df_WT_onlyIV40Short_rec_for_analysis.xlsx")
#filename, group, group_index, series, series_index are indentifier of recording
#Vmin, Vmax define the range of potentials in which the data will be analyzed
directory_df

In [ ]:
# selecting the file by index, that should be analyzed
index=1
directory_df.loc[index]

In [ ]:
# create an array with the potential range in which the data is to be analyzed
pot=[x / 1000.0 for x in range(-55, 55, 10)]
pot


In [ ]:
# get I_pA values either by fit or calculating the mean for selected potential traces
data_I=fit_exp_get_min(directory_df, start=100, stop=5000, step = 50, indices=[index], pot=pot, calc_mean=False, bin=True)
